# Self-Consistency Algorithm Demo
This notebook demonstrates the Self-Consistency algorithm for mathematical reasoning.

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import os
from dotenv import load_dotenv
from its_hub.utils import SAL_STEP_BY_STEP_SYSTEM_PROMPT
from its_hub.lms import OpenAICompatibleLanguageModel
import nest_asyncio
nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv()

# Main example: OpenAI API endpoint with gpt-4o-mini
lm = OpenAICompatibleLanguageModel(
    endpoint="https://api.openai.com/v1", 
    api_key=os.getenv("OPENAI_API_KEY"),  # Load API key from environment
    model_name="gpt-4o-mini", 
    system_prompt=SAL_STEP_BY_STEP_SYSTEM_PROMPT, 
    is_async=True,
)

/var/folders/5q/lcfzmsbs043d9666kwj77my00000gn/T/ipykernel_70166/3682206950.py:12: DeprecationWarning: The 'is_async' parameter is deprecated and will be removed in a future version. The implementation now always uses async internally. Sync methods (generate, evaluate) automatically wrap async calls with asyncio.run().
  lm = OpenAICompatibleLanguageModel(


In [9]:
# Alternative: vLLM local endpoint (commented out)
# lm = OpenAICompatibleLanguageModel(
#     endpoint="http://localhost:8000/v1", 
#     api_key="NO_API_KEY", 
#     model_name="qwen2-math-1.5b-instruct", 
#     system_prompt=SAL_STEP_BY_STEP_SYSTEM_PROMPT, 
#     is_async=True,
# )

In [25]:
# Mathematical problem to solve
prompt = r"Let $a$ be a positive real number such that all the roots of \[x^3 + ax^2 + ax + 1 = 0\]are real. Find the smallest possible value of $a.$"

# Generate response using the proper format
from its_hub.types import ChatMessages
chat_messages = ChatMessages.from_prompt_or_messages(prompt)
response = lm.generate(chat_messages.to_batch(1))[0]

print(response)

{'role': 'assistant', 'content': "To ensure all roots of the polynomial \\( x^3 + ax^2 + ax + 1 = 0 \\) are real, we will apply the criteria for real roots using the properties of the discriminant and the derivative to analyze critical points.\n\n## Step 1: Find the derivative\nFirst, we compute the derivative of the polynomial:\n\\[\nf(x) = x^3 + ax^2 + ax + 1.\n\\]\nThe derivative \\( f'(x) \\) is:\n\\[\nf'(x) = 3x^2 + 2ax + a.\n\\]\nThe critical points occur where \\( f'(x) = 0 \\):\n\\[\n3x^2 + 2ax + a = 0.\n\\]\nWe can use the discriminant condition to ensure real roots for \\( f'(x) \\):\n\\[\nD' = (2a)^2 - 4 \\cdot 3 \\cdot a = 4a^2 - 12a.\n\\]\nFor \\( f'(x) \\) to have real roots, the discriminant must be non-negative:\n\\[\n4a^2 - 12a \\geq 0.\n\\]\nFactoring gives:\n\\[\n4a(a - 3) \\geq 0.\n\\]\nThe solutions are \\( a \\leq 0 \\) or \\( a \\geq 3 \\). Since \\( a \\) must be positive, we conclude:\n\\[\na \\geq 3.\n\\]\n\n## Step 2: Evaluate \\( a = 3 \\) \nNext, we check t

In [26]:
def extract_boxed(s: str) -> str:
    import re
    # find all occurrences of \boxed{...}
    boxed_matches = re.findall(r'\\boxed\{([^{}]+(?:\{[^{}]*\}[^{}]*)*)\}', s)
    # return the last match if any were found
    return boxed_matches[-1] if boxed_matches else ""
    
print(extract_boxed(response['content']))

3


## Self-Consistency Algorithm
Now we'll use the Self-Consistency algorithm to improve the answer quality.

In [27]:
from its_hub.algorithms import SelfConsistency

# Set computational budget for scaling
budget = 4

scaling_alg = SelfConsistency(extract_boxed)

scaling_result = scaling_alg.infer(
    lm, prompt, budget, return_response_only=False
)

print("######## Self-Consistency Result ########")
print(scaling_result.the_one)

######## Self-Consistency Result ########
{'role': 'assistant', 'content': "To find the smallest possible value of \\(a\\) such that all the roots of the polynomial \n\n\\[\nf(x) = x^3 + ax^2 + ax + 1 \n\\]\n\nare real, we can use the condition on the discriminant for a cubic polynomial. Alternatively, we can apply Descartes' Rule of Signs or analyze the roots directly.\n\nLet's rewrite the polynomial:\n\n\\[\nf(x) = x^3 + ax^2 + ax + 1.\n\\]\n\nFor all roots to be real, \\(f(x)\\) must have a certain shape. We begin by analyzing the behavior of \\(f(x)\\). \n\n## Step 1: Critical points\nFinding the critical points is essential to understand the behavior of \\(f(x)\\). We compute the derivative:\n\n\\[\nf'(x) = 3x^2 + 2ax + a.\n\\]\n\nSetting the derivative to zero to find critical points:\n\n\\[\n3x^2 + 2ax + a = 0.\n\\]\n\nUsing the quadratic formula, the critical points are:\n\n\\[\nx = \\frac{-2a \\pm \\sqrt{(2a)^2 - 4 \\cdot 3 \\cdot a}}{2 \\cdot 3} = \\frac{-2a \\pm \\sqrt{4a^2 

In [28]:
print("######## Extracted Response Counts ########")
print(scaling_result.response_counts)

######## Extracted Response Counts ########
Counter({'3': 4})


## Self-Consistency Algorithm for Tool Calls
We have hierarchical tool-voting support in Self-Consistency algorithm
It first votes on tool names, and then on tool arguments.

In [29]:
from its_hub.types import ChatMessage, ChatMessages

# Tool schema (OpenAI-style dicts)
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Perform arithmetic calculations",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to evaluate"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

# ChatMessages instance with system + user
tool_call_messages = ChatMessages([
    ChatMessage(
        role="system",
        content="You are a precise calculator. Always use the calculator tool for arithmetic and format your final answer as \\boxed{result}."
    ),
    ChatMessage(
        role="user",
        content="What is 847 * 293 + 156?"
    ),
])

In [30]:
# Use hierarchical tool voting
scaling_alg_tool = SelfConsistency(tool_vote="tool_hierarchical")

budget = 5
scaling_result = scaling_alg_tool.infer(
    lm, tool_call_messages, budget, return_response_only=False, tools=tools, tool_choice="auto"
)

In [31]:
print("######## Self-Consistency Result ########")
print(scaling_result.the_one)

print("######## Tool Call Response Counts ########")
print(scaling_result.response_counts)

######## Self-Consistency Result ########
{'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_oFWhTYIFsIyD8kFzDboqlVUr', 'type': 'function', 'function': {'name': 'calculator', 'arguments': '{"expression": "847 * 293"}'}}, {'id': 'call_vxOtcx0wbr7T78JSLBZgHO0J', 'type': 'function', 'function': {'name': 'calculator', 'arguments': '{"expression": "156"}'}}], 'refusal': None, 'annotations': []}
######## Tool Call Response Counts ########
Counter({('calculator', (('expression', '847 * 293'),)): 5})
